# Gradient Descent with Minibatch and Momentum

In [55]:
import numpy as np

class MomentumGradientDescent:
    def __init__(
        self,
        learning_rate=0.001,
        momentum=0.9,
        max_iters=1e4,
        epsilon=1e-8,
        batch_size=32,
        record_history=False,
    ):
        self.learning_rate = learning_rate
        self.momentum = momentum
        self.max_iters = max_iters
        self.record_history = record_history
        self.epsilon = epsilon
        self.batch_size = batch_size
        self.prev_delta_w = None
        if record_history:
            # to store the weight history for visualization
            self.w_history = []

    def run(self, gradient_fn, x, y, w):
        grad = np.inf
        t = 1
        N, D = x.shape
        self.prev_delta_w = np.zeros(w.shape)
        while np.linalg.norm(grad) > self.epsilon and t < self.max_iters:
            for i in range(0, N, self.batch_size):
                if x.ndim == 1:
                    batch_x = x[i:i + self.batch_size]
                else:
                    batch_x = x[i:i + self.batch_size, :]

                if y.ndim == 1:
                    batch_y = y[i:i + self.batch_size]
                else:
                    batch_y = y[i:i + self.batch_size, :]

                # compute the gradient with present weight
                grad = gradient_fn(batch_x, batch_y, w)
                delta_w = self.get_delta_w(grad)

                # weight update step
                w = w - self.learning_rate * delta_w
                if self.record_history:
                    self.w_history.append(w)
            t += 1
        return w

    def get_delta_w(self, grad):
        beta = self.momentum
        delta_w = beta * self.prev_delta_w + (1 - beta) * grad
        self.prev_delta_w = delta_w

        return delta_w

# Softmax Regression

In [98]:
import numpy as np

# from the given Colab code
logistic = lambda z: 1./ (1 + np.exp(-z))  

class SoftmaxRegression:

    def __init__(self, add_bias=True):
        self.add_bias = add_bias
        pass
            
    def fit(self, x, y, C, optimizer):
        if x.ndim == 1:
            x = x[:, None]
        if self.add_bias:
            N = x.shape[0]
            x = np.column_stack([x,np.ones(N)])
        N,D = x.shape
        
        def to_onehot(a):
            return np.eye(C)[a]
        
        def gradient(x, y, w):
            N, D = x.shape
            # yh: N x C
            yh = self.softmax(np.dot(x, w))
            # both are N x C
            yh = to_onehot(self.to_classlabel(yh))
            y = to_onehot(y)
            
            grad = np.dot(x.T, yh - y) / N
            return grad
        
        # initialize all weights to 0
        w0 = np.zeros((D,C)) 
        # run the optimizer to get the optimal weights
        self.w = optimizer.run(gradient, x, y, w0) 
        return self
    
    def softmax(self, z):
        # to prevent overflow/underflow
        z = z - np.max(z, axis=-1, keepdims=True)
        return (np.exp(z.T) / np.sum(np.exp(z), axis=1)).T

    def to_classlabel(self, z):
        return z.argmax(axis=1)
    
    def predict(self, x):
        if self.add_bias:
            x = np.column_stack([x,np.ones(N)])
        # convert from 1D to 2D
        x = np.reshape(x, (1, -1))
        yh = self.softmax(np.dot(x, self.w))
        return self.to_classlabel(yh)[0]
        

# Validation of Model

Validation is performed using K-fold cross-validation. 

In [97]:
import math
import pandas as pd 

# Returns 2 datasets (training and validation)
def k_fold_splitter(fold, dataset):
    start = math.floor(fold*(dataset.shape[0]/5))
    end = math.floor((fold+1)*(dataset.shape[0]/5))

    training = np.delete(dataset, slice(start, end), axis=0)
    validation = dataset[start:end-1]

    return training, validation

def calculate_model_accuracy(x, y, C, learning_rate, momentum, batch_size):
    accuracies = []
    # do 5-fold cross-validation
    for fold_num in range(5):
        train_data, validation_data = k_fold_splitter(fold_num, x)
        train_labels, validation_labels = k_fold_splitter(fold_num, y)

        optimizer = MomentumGradientDescent(
            learning_rate=learning_rate, 
            momentum=momentum, 
            batch_size=batch_size, 
            max_iters=10000,
        )
        model = SoftmaxRegression(add_bias=False)
        model.fit(train_data, train_labels, C, optimizer)

        num_misclassified = 0
        # calculate the accuracy
        for i in range(len(validation_data)):
            prediction = model.predict(validation_data[i, :])
            if prediction != validation_labels[i]:
                num_misclassified += 1

        misclassification_rate = num_misclassified / len(validation_labels)
        accuracies.append(1 - misclassification_rate)
        
    return np.average(accuracies)
    

# Digits Dataset

First we do a grid search to find a good set of hyper-parameters (learning rate, momentum, batch size). This will give us a reference point for the model's performance which we can compare to as we change individual hyper-parameters. These parameters are not optimal since our grid search does not try an extensive set of combinations, but they are at least a good starting point. We kept the number of combinations on the smaller side since each loop iteration is fairly expensive in terms of computation time.

In [108]:
from sklearn.datasets import load_digits

digits = load_digits()
x, y = digits.data, digits.target

C = 10
max_accuracy = 0
best_learning_rate = 0
best_momentum = 0
best_batch_size = x.shape[0]

# grid search to find good hyper-parameters
for learning_rate in np.linspace(0.0001, 0.2, 21):
    for momentum in np.linspace(0.5, 0.99, 11):
        accuracy = calculate_model_accuracy(x, y, C, learning_rate, momentum, x.shape[0])
        if accuracy > max_accuracy:
            print(accuracy, end = ', ')
            max_accuracy = accuracy
            best_learning_rate = learning_rate
            best_momentum = momentum
            
for batch_size in range(1, x.shape[0], 64):
    accuracy = calculate_model_accuracy(x, y, C, best_learning_rate, best_momentum, batch_size)
    if accuracy > max_accuracy:
        print(accuracy, end = ' [BATCH], ')
        max_accuracy = accuracy
        best_batch_size = batch_size
            
print("\n\nBest Accuracy: {}".format(max_accuracy))  
print("Best Learning Rate: {}".format(best_learning_rate))  
print("Best Momentum: {}".format(best_momentum))  
print("Best Batch Size: {}".format(best_batch_size))  

0.9179642395854406, 0.9213224195079441, 0.9218873033410622 [BATCH], 0.9224490748665598 [BATCH], 

Best Accuracy: 0.9224490748665598
Best Learning Rate: 0.0001
Best Momentum: 0.549
Best Batch Size: 321


# Wine Dataset

Similar to above, we first do a grid search to find a good set of hyper-parameters as a reference point for the model's performance.

In [109]:
from sklearn.datasets import load_wine

wine = load_wine()
x, y = wine.data, wine.target

C = 3
max_accuracy = 0
best_learning_rate = 0
best_momentum = 0
best_batch_size = x.shape[0]

# grid search to find good hyper-parameters
for learning_rate in np.linspace(0.0001, 0.2, 21):
    for momentum in np.linspace(0.5, 0.99, 11):
        accuracy = calculate_model_accuracy(x, y, C, learning_rate, momentum, x.shape[0])
        if accuracy > max_accuracy:
            print(accuracy, end=', ')
            max_accuracy = accuracy
            best_learning_rate = learning_rate
            best_momentum = momentum
            
for batch_size in range(1, x.shape[0], 64):
    accuracy = calculate_model_accuracy(x, y, C, best_learning_rate, best_momentum, batch_size)
    if accuracy > max_accuracy:
        print(accuracy, end=' [BATCH], ')
        max_accuracy = accuracy
        best_batch_size = batch_size
            
print("\n\nBest Accuracy: {}".format(max_accuracy))
print("Best Learning Rate: {}".format(best_learning_rate))
print("Best Momentum: {}".format(best_momentum))
print("Best Batch Size: {}".format(best_batch_size))

0.6726050420168067, 0.712436974789916, 0.9124369747899159, 0.9188235294117648, 

Best Accuracy: 0.9188235294117648
Best Learning Rate: 0.0001
Best Momentum: 0.892
Best Batch Size: 178


# Comparison Against Other Classifiers

Now we can compare our model to an off-the-shelf classifier to see how the performance compares on the digits dataset. We will be comparing against the Ridge Regression and Logistic Regression models from SciKit Learn.

### Digits Dataset Comparison

In [92]:
from sklearn.datasets import load_digits
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression

digits = load_digits()
x, y = digits.data, digits.target

C = 10
# using the hyper-parameters obtained from grid search
softmax_accuracy = calculate_model_accuracy(x, y, C, 0.0001, 0.549, 321)

# Ridge Regression
ridge_model_accuracies = []
for fold_num in range(5):
    train_data, validation_data = k_fold_splitter(fold_num, x)
    train_labels, validation_labels = k_fold_splitter(fold_num, y)

    ridge_model = RidgeClassifier(max_iter=10000)
    ridge_model.fit(train_data, train_labels)

    ridge_model_accuracies.append(ridge_model.score(validation_data, validation_labels))
    
# Logistic Regression
logistic_model_accuracies = []
for fold_num in range(5):
    train_data, validation_data = k_fold_splitter(fold_num, x)
    train_labels, validation_labels = k_fold_splitter(fold_num, y)

    logistic_model = LogisticRegression(multi_class="multinomial", max_iter=10000)
    logistic_model.fit(train_data, train_labels)

    logistic_model_accuracies.append(logistic_model.score(validation_data, validation_labels))

print("Softmax Regression Accuracy: {}".format(softmax_accuracy))
print("Ridge Regression (SKLearn) Accuracy: {}".format(np.average(ridge_model_accuracies)))
print("Logistic Regression (SKLearn) Accuracy: {}".format(np.average(logistic_model_accuracies)))

Softmax Regression Accuracy: 0.9224490748665598
Ridge Regression (SKLearn) Accuracy: 0.8900795194597034
Logistic Regression (SKLearn) Accuracy: 0.9179657957392509


### Wine Dataset Comparison

We can do the same comparison on the wine dataset. This is done below.

In [110]:
from sklearn.datasets import load_wine
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import LogisticRegression

wine = load_wine()
x, y = wine.data, wine.target

C = 3
# using the hyper-parameters obtained from grid search
softmax_accuracy = calculate_model_accuracy(x, y, C, 0.0001, 0.892, 178)

# Ridge Regression
ridge_model_accuracies = []
for fold_num in range(5):
    train_data, validation_data = k_fold_splitter(fold_num, x)
    train_labels, validation_labels = k_fold_splitter(fold_num, y)

    ridge_model = RidgeClassifier(max_iter=10000)
    ridge_model.fit(train_data, train_labels)

    ridge_model_accuracies.append(ridge_model.score(validation_data, validation_labels))
    
# Logistic Regression
logistic_model_accuracies = []
for fold_num in range(5):
    train_data, validation_data = k_fold_splitter(fold_num, x)
    train_labels, validation_labels = k_fold_splitter(fold_num, y)

    logistic_model = LogisticRegression(multi_class="multinomial", max_iter=10000)
    logistic_model.fit(train_data, train_labels)

    logistic_model_accuracies.append(logistic_model.score(validation_data, validation_labels))

print("Softmax Regression Accuracy: {}".format(softmax_accuracy))
print("Ridge Regression (SKLearn) Accuracy: {}".format(np.average(ridge_model_accuracies)))
print("Logistic Regression (SKLearn) Accuracy: {}".format(np.average(logistic_model_accuracies)))

Softmax Regression Accuracy: 0.9188235294117648
Ridge Regression (SKLearn) Accuracy: 0.9478991596638655
Logistic Regression (SKLearn) Accuracy: 0.9359663865546219
